In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import numpy as np
from scipy import stats

In [2]:
def regress(X,Y):
    X_cp = sm.add_constant(X)
    model = sm.OLS(Y,X_cp)
    results = model.fit()
    return results

In [3]:
def bestNvariables(df, listX, y,n=2):
    '''
    Escolhe as n melhores variaveis para prever uma variavel dependente de forma
    linear e retorna uma regressão linear.
    
    df --> DataFrame
    listX --> Lista de variaveis independentes
    y --> Variavel dependente/a ser analizada 
    n --> quantidade final das variaveis, 2 como padrão
    '''
    y = df[[y]]
    xs = df[listX]
    results = regress(xs, y)
    
    while len(results.pvalues) > n+1:
        actual_max = 0
        for e in list(round(results.pvalues,10)):
            if e > actual_max:
                actual_max = e
        j = 0
        for e in list(round(results.pvalues,10)):
            j += 1
            if e == actual_max:
                break
        listX.remove(listX[j-2])
        xs = df[listX]
        results = regress(xs, y)
    return results

In [4]:
def variablesLessAlpha(df, listX, y, alpha=0.05):
    '''
    Exclui todas as variaveis em que o P>|t| for maior que o alpha e retorna uma regressão linear
    
    df --> DataFrame
    listX --> Lista de variaveis independentes
    y --> Variavel dependente/a ser analizada 
    alpha --> nivel de significancia, 5% como padrão'''
    
    y = df[[y]]
    xs = df[listX]
    results = regress(xs, y)

    while actual_max > alpha:
        actual_max = max(results.pvalues)
        j = 0
        if actual_max > alpha:
            for e in list(results.pvalues):
                j += 1
                if e == actual_max:
                    break
            listX.remove(listX[j-2])
            xs = df[listX]
            results = regress(xs, y)
            
    return results

In [5]:
def Boostrap_media(listX, dobs, quant_permut=10000, alpha=5, bicaudal=True, grid=False, legend=True, dobs_legend="Media observada", critico_legend="Xcritico"):
    '''
    Aplica bootstrap em listX e plota o histograma da distribuição das medias
    
    listX --> lista a ser analizada
    dobs --> media experimental
    quant_permut --> quantidade de permutaçoes a serem realizadas, padrão:10000
    alpha --> nivel de significancia. de 0 a 100%
    bicaudal --> se é bicaudal ou unicaudal
    grid --> se coloca grid
    legend --> se coloca legenda
    dobs_legend --> legenda do dobs. padrão: Media esperada
    critico_legend --> legenda do xcritico. padrão: Xcritico
    
    '''
    n = len(listX)
    x_mean = []
    for e in range(quant_permut):
        reamostra_x = np.random.choice(x, size=n, replace=True)
        x_mean.append(np.mean(reamostra_x))
    if bicaudal:
        plt.hist(x_mean, bins=50);
        plt.axvline(dobs, color='black', label=dobs_legend)
        plt.axvline(np.percentile(x_mean, 100-alpha/2), color='red', label=critico_legend)
        plt.axvline(np.percentile(x_mean, alpha/2), color='red')
    else:
        plt.hist(x_mean, bins=50);
        plt.axvline(dobs, color='black')
        plt.axvline(np.percentile(x_mean, 100-alpha/2), color='red')
    if legend:
        plt.legend(legend)
    plt.gride(grid)
    plt.show()
        

In [6]:
def Boostrap_var(listX, dobs, quant_permut=10000, alpha=5, bicaudal=True, grid=False, legend=True, dobs_legend="Variancia esperada", critico_legend="Xcritico"):
    '''
    Aplica bootstrap em listX e plota o histograma da distribuição das medias
    
    listX --> lista a ser analizada
    dobs --> media experimental
    quant_permut --> quantidade de permutaçoes a serem realizadas, padrão:10000
    alpha --> nivel de significancia. de 0 a 100%
    bicaudal --> se é bicaudal ou unicaudal
    grid --> se coloca grid
    legend --> se coloca legenda
    dobs_legend --> legenda do dobs. padrão: Variancia esperada
    critico_legend --> legenda do xcritico. padrão: Xcritico
    
    '''
    n = len(listX)
    x_var = []
    for e in range(quant_permut):
        reamostra_x = np.random.choice(x, size=n, replace=True)
        x_var.append(np.var(reamostra_x))
    if bicaudal:
        plt.hist(x_var, bins=50);
        plt.axvline(dobs, color='black', label=dobs_legend)
        plt.axvline(np.percentile(x_var, 100-alpha/2), color='red', label=critico_legend)
        plt.axvline(np.percentile(x_var, alpha/2), color='red')
    else:
        plt.hist(x_var, bins=50);
        plt.axvline(dobs, color='black')
        plt.axvline(np.percentile(x_var, 100-alpha/2), color='red')
    if legend:
        plt.legend()
    plt.gride(grid)
    plt.show()
    

In [7]:
def AmostraIndependenteMedia(xA, xB, alpha=5, n=10000, grid=False, legend=True, dobs_legend="Delta Media Inicial", critico_legend="Xcritico", bins=20):
    '''
    '''
    
    xAB = xA + xB
    deltaMean = []
    
    dobs = np.mean(xB)-np.mean(xA)

    for e in range(n):
        np.random.shuffle(xAB)

        shuffleA = xAB[0:len(xA)+1]
        shuffleB = xAB[len(xA)+1:]

        meanShuffleA = np.mean(shuffleA)
        meanShuffleB = np.mean(shuffleB)

        deltaMean.append(meanShuffleB-meanShuffleA)
    
    XC1 = np.percentile(deltaMean, alpha/2)
    XC2 = np.percentile(deltaMean, 100-alpha/2)
    
    
    plt.hist(deltaMean, bins=bins);
    plt.axvline(dobs, color='black', label=dobs_legend)
    plt.axvline(-dobs, color='black', label='menos '+dobs_legend)
    plt.axvline(XC1, color='red', label=critico_legend)
    plt.axvline(XC2, color='red')
    if legend:
        plt.legend()
    plt.grid(grid)
    plt.show()
    
    vezes = 0
    for e in deltaMean:
        if e > dobs or e < -dobs:
            vezes += 1
    p_value = vezes/n
    
    return p_value
    
    

In [8]:
def AmostraDependenteCorr(x, y, alpha=5, n=10000, grid=False, legend=True, robs_legend="Variancia esperada", critico_legend="Xcritico", bins=50):
    '''
    '''
    corrList = []
    
    robs = np.corrcoef(x,y)[0,1]

    for e in range(n):
        np.random.shuffle(y)
        corr = np.corrcoef(x,y)[0,1]
        corrList.append(corr)
    
    XC1 = np.percentile(corrList, alpha/2)
    XC2 = np.percentile(corrList, 100-alpha/2)
    
    plt.hist(corrList, bins=bins);
    plt.axvline(robs, color='black', label=robs_legend)
    plt.axvline(-robs, color='black', label='menos '+robs_legend)
    plt.axvline(XC1, color='red', label=critico_legend)
    plt.axvline(XC2, color='red')
    if legend:
        plt.legend()
    plt.grid(grid)
    plt.show()
    
    vezes = 0
    for e in corrList:
        if e > robs or e < -robs:
            vezes += 1
    p_value = vezes/n
    p_value
    
    return p_value
    
    

In [9]:
def tStudent(amostra, H0, alpha=0.05, c='right', pvalue=True):
    '''
    '''
    if c != "both" and c != "right" and c != "right":
        return None
    
    n = len(amostra)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
    x_obs = np.mean(amostra)
    s = np.std(amostra, ddof=1)
    
    t_obs = (x_obs-H0)/(s/np.sqrt(n))
    t_c = stats.t.ppf(alpha, df=n-1)
    
    scale = s/n**(0.5)
    
    if pvalue:
        if c == "left":
            valor_p = stats.t.cdf(x_obs, df=n-1, loc=H0, scale=scale)

        if c == "right":
            valor_p = 1-stats.t.cdf(x_obs, df=n-1, loc=H0, scale=scale)
            
        if c == 'both':
            if x_obs < H0:
                valor_p = stats.t.cdf(x_obs, df=n-1, loc=H0, scale=scale)*2
            else:
                valor_p = (1-stats.t.cdf(x_obs, df=n-1, loc=H0, scale=scale))*2
                
        return valor_p
    
    else:
        if c == "left":
            XC = stats.t.ppf(alpha, loc=H0, scale=scale)

        if c == "right":
            XC = stats.t.ppf(1-alpha, loc=H0, scale=scale)

        return XC

        if c == "both":
            XC1 = stats.t.ppf(alpha/2, loc=H0, scale=scale)
            XC2 = stats.t.ppf(1-alpha/2, loc=H0, scale=scale)
            return XC1, XC2

In [35]:
def combUniformExpo(loc_unif=0, unifx1=0, unifx2=1, scale_exp=1, n=5000, x0=-1, x1=15, qtp=300, bins=50):
    '''
    '''
    eixo_x = np.linspace(x0, x1, qtp)
    
    scale_unif = unifx2-unifx1
    pdf_unif = stats.uniform.pdf(eixo_x, loc=loc_unif, scale=scale_unif)
    
    pdf_exp = stats.expon.pdf(eixo_x, scale=scale_exp)
    
    x_ = stats.uniform.rvs(loc=loc_unif, scale=scale_unif, size=n)
    y_ = stats.expon.rvs(scale=scale_exp, size=n)
    z_ = x_ + y_
        
    plt.hist(z_, label="z", bins=bins, density=True)
    plt.plot(eixo_x, pdf_unif, label="uniforme")
    plt.plot(eixo_x, pdf_exp, label="exponencial")
    plt.legend(loc='upper right');